In [ ]:
import os
import pandas as pd
import numpy as np
import string
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score, silhouette_score, davies_bouldin_score
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN, HDBSCAN
from sklearn.manifold import TSNE
import umap
import warnings
warnings.filterwarnings('ignore')

nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
# Mount Google Drive (required every time)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define and check the paths
# PROJECT_ROOT assumes the shared Milestone II folder is in your root google drive
PROJECT_ROOT = '/content/drive/MyDrive/SIADS 692 Milestone II/Milestone II - NLP Cryptic Crossword Clues' # Nathan's Drive
DATA_DIR = f"{PROJECT_ROOT}/data"
NOTEBOOK_DIR = f"{PROJECT_ROOT}/notebooks"
OUTPUT_DIR = f"{PROJECT_ROOT}/outputs"
IND_DIR = f'{DATA_DIR}/verified_indicators.csv'

if not os.path.exists(PROJECT_ROOT):
    PROJECT_ROOT = os.path.abspath("..")  # fallback for local runs

What I'd Actually Recommend 🎯
For your 15k wordplay terms:

- Start with this analysis to get a ballpark (15-40 clusters seems reasonable)
- Use DBSCAN instead of k-means - it can find variable-density clusters and label outliers/noise
- Try hierarchical clustering with a dendrogram - this is probably better for wordplay since categories naturally nest (e.g., "phonetic wordplay" → "rhymes" → "perfect rhymes" vs "slant rhymes")
- Sample and validate - Pick 5-10 clusters, look at 20 random words from each, ask "do these belong together?"
- Consider manual seeding - If you have examples of clear categories, use semi-supervised clustering

In [ ]:
import os
import pandas as pd
import numpy as np
import string
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score, silhouette_score, davies_bouldin_score
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN, HDBSCAN
from sklearn.manifold import TSNE
import umap
import warnings
warnings.filterwarnings('ignore')

nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')


"""
Embedding Model Comparison for Wordplay Clustering
Compares multiple embedding models and creates visualizations
"""

class EmbeddingComparison:
    """Compare different embedding models for clustering wordplay terms"""

    def __init__(self, words, output_dir=OUTPUT_DIR):
        self.words = words
        self.output_dir = output_dir
        self.results = {}
        os.makedirs(self.output_dir, exist_ok=True)

    def get_sentence_transformer_embeddings(self, model_name):
        """Get embeddings using sentence-transformers"""
        print(f"Loading {model_name}...")
        model = SentenceTransformer(model_name)
        print(f"Generating embeddings for {len(self.words)} words...")
        embeddings = model.encode(self.words, show_progress_bar=True, batch_size=32)
        embeddings = normalize(embeddings)
        return embeddings

    def reduce_dimensions(self, embeddings, n_components=100):
        print(f"🔹 Applying PCA for clustering: {n_components} dims")

        pca = PCA(n_components=n_components, random_state=42)
        reduced = pca.fit_transform(embeddings)

        # re-normalize after PCA
        reduced = normalize(reduced)

        explained = np.sum(pca.explained_variance_ratio_)
        print(f"   → Variance retained: {explained:.2%}")

        return reduced

    def cluster_embeddings(self, embeddings, n_clusters=10, method='kmeans'):
        """Cluster embeddings using specified method"""
        if method == 'kmeans':
            clusterer = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
            labels = clusterer.fit_predict(embeddings)
        elif method == 'dbscan':
            clusterer = DBSCAN(eps=0.5, min_samples=5, metric='cosine')
            labels = clusterer.fit_predict(embeddings)
        elif method == 'hierarchical':
            from scipy.cluster.hierarchy import linkage, fcluster
            from scipy.spatial.distance import squareform
            from sklearn.metrics.pairwise import cosine_distances

            # Compute distance matrix
            distances = cosine_distances(embeddings)
            condensed_distances = squareform(distances)

            # Perform hierarchical clustering
            Z = linkage(condensed_distances, method='average')

            # Cut tree to get n_clusters
            labels = fcluster(Z, n_clusters, criterion='maxclust') - 1  # -1 to make 0-indexed
        else:
            raise ValueError(f"Unknown clustering method: {method}")

        return labels

    def _compute_centroids(self, embeddings, labels):
        """Helper to compute centroids of non-noise clusters."""
        centroids = []
        cluster_ids = sorted([l for l in set(labels) if l != -1])

        if not cluster_ids:  # Handle case where no non-noise clusters exist
            return None, None  # Return None for centroids and cluster_ids

        for lab in cluster_ids:
            centroids.append(embeddings[labels == lab].mean(axis=0))

        return np.vstack(centroids), cluster_ids


    def auto_dbscan(self, embeddings, min_samples=5, eps_range=None):
        """
        Automatically find good DBSCAN parameters using silhouette score
        NC 2/7/26, added compute_centroids and noise points reassignment

        Args:
            embeddings: Embedding vectors
            min_samples: Minimum samples per cluster
            eps_range: Range of eps values to try (default: np.arange(0.1, 1.0, 0.1))

        Returns:
            Best labels, best eps value, and metrics
        """
        if eps_range is None:
            #eps_range = np.arange(0.1, 1.0, 0.1)
            eps_range = np.arange(0.1, 0.5, 0.05)

        best_score = -1
        best_eps = None
        best_labels = None
        results = []

        print(f"  Testing DBSCAN with eps values: {list(eps_range)}")

        for eps in eps_range:
            clusterer = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine')
            labels = clusterer.fit_predict(embeddings)

            # Count clusters (excluding noise points labeled -1)
            n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            n_noise = list(labels).count(-1)

            # If no actual clusters are found, we cannot compute centroids or meaningful metrics.
            if n_clusters == 0:
                continue

            # Reassign ALL noise labels to closest centroid.
            new_labels = labels.copy()
            if n_noise > 0: # Only try to reassign if there are noise points
                centroids, cluster_ids = self._compute_centroids(embeddings, labels)
                for i, x in enumerate(embeddings):
                    if labels[i] == -1:   # was noise
                        sims = centroids @ x   # cosine similarity
                        new_labels[i] = cluster_ids[np.argmax(sims)]

            # Need at least 2 clusters for these metrics
            if n_clusters < 2:
                continue  # Need at least 2 clusters for metrics

            # Calculate silhouette for non-noise points
            mask = labels != -1
            if mask.sum() > 0:
                filtered_embeddings = embeddings[mask]
                filtered_labels = labels[mask]

                if len(set(filtered_labels)) >= 2:
                    silhouette = silhouette_score(filtered_embeddings, filtered_labels)

                    results.append({
                        'eps': eps,
                        'n_clusters': n_clusters,
                        'n_noise': n_noise,
                        'silhouette': silhouette
                    })

                    if silhouette > best_score:
                        best_score = silhouette
                        best_eps = eps
                        best_labels = new_labels # Store labels with reassigned noise

        if best_labels is None:
            print("  Warning: Could not find good DBSCAN parameters, using default eps=0.5")
            clusterer = DBSCAN(eps=0.5, min_samples=min_samples, metric='cosine')
            best_labels = clusterer.fit_predict(embeddings)
            best_eps = 0.5

        return best_labels, best_eps, results

    def evaluate_clustering(self, embeddings, labels):
        """Evaluate clustering quality"""
        # Filter out noise points (label = -1) for DBSCAN
        mask = labels != -1
        if not mask.any():
            return {'silhouette': None, 'davies_bouldin': None, 'n_clusters': 0}

        filtered_embeddings = embeddings[mask]
        filtered_labels = labels[mask]

        # Need at least 2 clusters for these metrics
        n_clusters = len(np.unique(filtered_labels))
        if n_clusters < 2:
            return {'silhouette': None, 'davies_bouldin': None, 'n_clusters': n_clusters}

        silhouette = silhouette_score(filtered_embeddings, filtered_labels)
        davies_bouldin = davies_bouldin_score(filtered_embeddings, filtered_labels)

        return {
            'silhouette': silhouette,
            'davies_bouldin': davies_bouldin,
            'n_clusters': n_clusters
        }

    def create_visualization(self, reduced_embeddings, labels, model_name,
                           reduction_method, clustering_method, metrics):
        """Create and save visualization"""
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))

        # Plot 1: Colored by cluster
        scatter1 = axes[0].scatter(
            reduced_embeddings[:, 0],
            reduced_embeddings[:, 1],
            c=labels,
            cmap='tab20',
            alpha=0.6,
            s=10
        )
        axes[0].set_title(
            f'{model_name}\n{reduction_method.upper()} + {clustering_method}',
            fontsize=14,
            fontweight='bold'
        )
        axes[0].set_xlabel(f'{reduction_method.upper()} 1')
        axes[0].set_ylabel(f'{reduction_method.upper()} 2')
        plt.colorbar(scatter1, ax=axes[0], label='Cluster')

        # Plot 2: Density plot
        axes[1].hexbin(
            reduced_embeddings[:, 0],
            reduced_embeddings[:, 1],
            gridsize=50,
            cmap='YlOrRd',
            mincnt=1
        )
        axes[1].set_title(
            f'{model_name}\nDensity Visualization',
            fontsize=14,
            fontweight='bold'
        )
        axes[1].set_xlabel(f'{reduction_method.upper()} 1')
        axes[1].set_ylabel(f'{reduction_method.upper()} 2')

        # Add metrics to plot
        metrics_text = f"Clusters: {metrics['n_clusters']}\n"
        if metrics['silhouette'] is not None:
            metrics_text += f"Silhouette: {metrics['silhouette']:.3f}\n"
            metrics_text += f"Davies-Bouldin: {metrics['davies_bouldin']:.3f}"
        else:
            metrics_text += "Metrics: N/A"

        fig.text(0.5, 0.02, metrics_text, ha='center', fontsize=12,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        plt.tight_layout()
        plt.subplots_adjust(bottom=0.1)

        # Save figure
        safe_name = model_name.replace('/', '_').replace(' ', '_')
        filename = f'{safe_name}_{reduction_method}_{clustering_method}.png'
        filepath = f'{self.output_dir}/{filename}'
        plt.savefig(filepath, dpi=150, bbox_inches='tight')
        plt.close()

        return filepath

    def analyze_model(self, model_name, embeddings=None,
                     #reduction_methods=['umap', 'tsne'],
                     reduction_methods=['PCA'],
                     clustering_methods=['kmeans'],
                     n_clusters=10):
        """Complete analysis pipeline for one model"""
        print(f"\n{'='*60}")
        print(f"Analyzing: {model_name}")
        print(f"{'='*60}")

        if embeddings is None:
            embeddings = self.get_sentence_transformer_embeddings(model_name)

        model_results = {
            'embeddings': embeddings,
            'visualizations': []
        }

        for reduction in reduction_methods:
            reduced = self.reduce_dimensions(embeddings)

            for clustering in clustering_methods:
                labels = self.cluster_embeddings(
                    embeddings,
                    n_clusters=n_clusters,
                    method=clustering
                )

                metrics = self.evaluate_clustering(embeddings, labels)

                viz_path = self.create_visualization(
                    reduced, labels, model_name,
                    reduction, clustering, metrics
                )

                model_results['visualizations'].append({
                    'reduction': reduction,
                    'clustering': clustering,
                    'metrics': metrics,
                    'path': viz_path
                })

                print(f"  {reduction} + {clustering}: {metrics}")

        self.results[model_name] = model_results
        return model_results

    def compare_cluster_counts(self, model_names, cluster_range, reduction_method='umap'):
        """
        Compare models across different cluster counts

        Args:
            model_names: List of model names to compare
            cluster_range: List or range of cluster counts to try
            reduction_method: Which dimensionality reduction to use ('umap' or 'tsne')

        Returns:
            DataFrame with metrics for each model and cluster count
        """
        results_data = []

        for model_name in model_names:
            print(f"\n{'='*60}")
            print(f"Testing {model_name} with varying cluster counts")
            print(f"{'='*60}")

            # Get or generate embeddings for this model
            if model_name in self.results and 'embeddings' in self.results[model_name]:
                embeddings = self.results[model_name]['embeddings']
                print(f"Using cached embeddings for {model_name}")
            else:
                embeddings = self.get_sentence_transformer_embeddings(model_name)
                if model_name not in self.results:
                    self.results[model_name] = {}
                self.results[model_name]['embeddings'] = embeddings

            # Test each cluster count
            for n_clusters in cluster_range:
                print(f"  Testing with {n_clusters} clusters...")

                labels = self.cluster_embeddings(
                    embeddings,
                    n_clusters=n_clusters,
                    method='kmeans'
                )

                metrics = self.evaluate_clustering(embeddings, labels)

                results_data.append({
                    'Model': model_name,
                    'N_Clusters': n_clusters,
                    'Silhouette': metrics['silhouette'],
                    'Davies_Bouldin': metrics['davies_bouldin'],
                    'Actual_Clusters': metrics['n_clusters']
                })

        df = pd.DataFrame(results_data)

        # Save to CSV
        csv_path = f'{self.output_dir}/cluster_count_comparison.csv'
        df.to_csv(csv_path, index=False)

        # Create visualization
        viz_path = self.visualize_cluster_comparison(df)

        return df, csv_path, viz_path

    def find_and_export_optimal_clusters(self, model_names=None, cluster_range=None,
                                        methods=['kmeans', 'dbscan', 'hierarchical']):
        """
        Find optimal cluster configuration for each model and export members

        Args:
            model_names: List of models to analyze (uses all cached if None)
            cluster_range: Range of cluster counts to test for kmeans/hierarchical
            methods: Clustering methods to try

        Returns:
            Dictionary with results for each model and method
        """
        if model_names is None:
            model_names = list(self.results.keys())

        if cluster_range is None:
            cluster_range = range(5, 31, 5)

        optimal_results = {}

        for model_name in model_names:
            print(f"\n{'='*70}")
            print(f"Finding optimal clusters for {model_name}")
            print(f"{'='*70}")

            # Get embeddings
            if model_name in self.results and 'embeddings' in self.results[model_name]:
                embeddings = self.results[model_name]['embeddings']
            else:
                print(f"  Generating embeddings...")
                embeddings = self.get_sentence_transformer_embeddings(model_name)
                if model_name not in self.results:
                    self.results[model_name] = {}
                self.results[model_name]['embeddings'] = embeddings

            model_results = {}

            # Test K-means
            if 'kmeans' in methods:
                print(f"\n  Testing K-means across {len(cluster_range)} cluster counts...")
                best_score = -1
                best_n = None
                best_labels = None

                for n in cluster_range:
                    labels = self.cluster_embeddings(embeddings, n_clusters=n, method='kmeans')
                    metrics = self.evaluate_clustering(embeddings, labels)

                    if metrics['silhouette'] and metrics['silhouette'] > best_score:
                        best_score = metrics['silhouette']
                        best_n = n
                        best_labels = labels

                print(f"  ✓ K-means optimal: {best_n} clusters (Silhouette: {best_score:.4f})")

                # Export members
                csv_path, txt_path = self.export_cluster_members(
                    f"{model_name}_kmeans_{best_n}",
                    best_labels
                )

                model_results['kmeans'] = {
                    'n_clusters': best_n,
                    'silhouette': best_score,
                    'labels': best_labels,
                    'csv_path': csv_path,
                    'txt_path': txt_path
                }

            # Test DBSCAN
            if 'dbscan' in methods:
                print(f"\n  Testing DBSCAN with auto-tuned parameters...")
                best_labels, best_eps, dbscan_results = self.auto_dbscan(embeddings)

                n_clusters = len(set(best_labels)) - (1 if -1 in best_labels else 0)
                n_noise = list(best_labels).count(-1)

                metrics = self.evaluate_clustering(embeddings, best_labels)

                print(f"  ✓ DBSCAN optimal: eps={best_eps:.2f}, {n_clusters} clusters, "
                      f"{n_noise} noise points (Silhouette: {metrics['silhouette']:.4f})")

                # Export members
                csv_path, txt_path = self.export_cluster_members(
                    f"{model_name}_dbscan_eps{best_eps:.2f}",
                    best_labels
                )

                model_results['dbscan'] = {
                    'eps': best_eps,
                    'n_clusters': n_clusters,
                    'n_noise': n_noise,
                    'silhouette': metrics['silhouette'],
                    'labels': best_labels,
                    'csv_path': csv_path,
                    'txt_path': txt_path,
                    'tuning_results': dbscan_results
                }

            # Test Hierarchical
            if 'hierarchical' in methods:
                print(f"\n  Testing Hierarchical clustering...")

                # Create dendrogram
                dendro_path = self.create_dendrogram(embeddings, model_name)

                # Find best cut
                best_score = -1
                best_n = None
                best_labels = None

                for n in cluster_range:
                    labels = self.cluster_embeddings(embeddings, n_clusters=n, method='hierarchical')
                    metrics = self.evaluate_clustering(embeddings, labels)

                    if metrics['silhouette'] and metrics['silhouette'] > best_score:
                        best_score = metrics['silhouette']
                        best_n = n
                        best_labels = labels

                print(f"  ✓ Hierarchical optimal: {best_n} clusters (Silhouette: {best_score:.4f})")

                # Export members
                csv_path, txt_path = self.export_cluster_members(
                    f"{model_name}_hierarchical_{best_n}",
                    best_labels
                )

                model_results['hierarchical'] = {
                    'n_clusters': best_n,
                    'silhouette': best_score,
                    'labels': best_labels,
                    'csv_path': csv_path,
                    'txt_path': txt_path,
                    'dendrogram_path': dendro_path
                }

            optimal_results[model_name] = model_results

            # Test HDBSCAN
            if 'hdbscan' in methods:
                print(f"\n  Testing HDBSCAN clustering...")

                def noise_aware_score(X, labels, alpha=1.0, beta=0.8):
                    # Use to penalize HDBSCAN models with high % of noise points.
                    # Increase beta to increase penalization.
                    # fraction of noise points
                    noise_pct = np.mean(labels == -1)

                    # compute silhouette only on non-noise points
                    mask = labels != -1
                    if len(set(labels[mask])) > 1:
                        sil = silhouette_score(X[mask], labels[mask])
                    else:
                        sil = -1

                    # combined objective
                    score = alpha * sil - beta * noise_pct
                    return score, sil, noise_pct

                best_score = -1
                best_params = None
                best_labels = None

                # Grid search over min_cluster_size and min_samples
                for min_cluster_size in [10, 15, 20]:
                    for min_samples in [1, 5, 10]:
                        print(f"Min size:{min_cluster_size}, min samples:{min_samples}")

                        clusterer = HDBSCAN(
                            min_cluster_size=min_cluster_size,
                            min_samples=min_samples,
                            metric='euclidean'
                        )
                        labels = clusterer.fit_predict(embeddings)

                        # Skip if all points are labeled as noise
                        if np.all(labels == -1):
                            continue

                        score, sil, noise_pct = noise_aware_score(embeddings, labels)

                        if score > best_score:
                            best_score = score
                            best_params = (min_cluster_size, min_samples)
                            best_labels = labels
                            best_sil = sil
                            best_noise = noise_pct

                if best_labels is None:
                    print("  ⚠️ HDBSCAN failed to produce any clusters.")
                else:
                    hdbscan_n_clusters = len(set(best_labels)) - (1 if -1 in best_labels else 0)
                    hdbscan_n_noise = list(best_labels).count(-1)
                    print(f"  ✓ HDBSCAN optimal: min_cluster_size={best_params[0]}, "
                          f"min_samples={best_params[1]} (Silhouette: {best_score:.4f})")

                    # Export members
                    csv_path, txt_path = self.export_cluster_members(
                        f"{model_name}_hdbscan_{best_params[0]}_{best_params[1]}",
                        best_labels
                    )

                    model_results['hdbscan'] = {
                        'min_cluster_size': best_params[0],
                        'min_samples': best_params[1],
                        'n_clusters': hdbscan_n_clusters,
                        'n_noise': hdbscan_n_noise,
                        'silhouette': best_score,
                        'labels': best_labels,
                        'csv_path': csv_path,
                        'txt_path': txt_path
                    }

            optimal_results[model_name] = model_results

        # Create summary report
        summary_path = self._create_optimal_clusters_summary(optimal_results)

        print(f"\n{'='*70}")
        print("OPTIMAL CLUSTER ANALYSIS COMPLETE!")
        print(f"{'='*70}")
        print(f"\nSummary report: {summary_path}")

        return optimal_results

    def _create_optimal_clusters_summary(self, optimal_results):
        """Create a summary report of optimal clustering results"""
        summary_path = f'{self.output_dir}/optimal_clusters_summary.txt'

        with open(summary_path, 'w', encoding='utf-8') as f:
            f.write("OPTIMAL CLUSTERING ANALYSIS SUMMARY\n")
            f.write("=" * 70 + "\n\n")

            for model_name, methods in optimal_results.items():
                f.write(f"MODEL: {model_name}\n")
                f.write("-" * 70 + "\n\n")

                for method, results in methods.items():
                    f.write(f"  {method.upper()}:\n")

                    if method == 'dbscan' or method == 'hdbscan': # Modified here
                        if method == 'hdbscan':
                            f.write(f"    min_cluster_size: {results['min_cluster_size']}\n")
                            f.write(f"    min_samples: {results['min_samples']}\n")
                        else: # dbscan
                            f.write(f"    eps: {results['eps']:.2f}\n")
                        f.write(f"    Clusters: {results['n_clusters']}\n")
                        f.write(f"    Noise points: {results['n_noise']}\n")
                    else:
                        f.write(f"    Optimal clusters: {results['n_clusters']}\n")

                    f.write(f"    Silhouette score: {results['silhouette']:.4f}\n")
                    f.write(f"    Full data: {results['csv_path'].split('/')[-1]}\n")
                    f.write(f"    Summary: {results['txt_path'].split('/')[-1]}\n")

                    if 'dendrogram_path' in results:
                        f.write(f"    Dendrogram: {results['dendrogram_path'].split('/')[-1]}\n")

                    f.write("\n")

                f.write("\n")

            # Recommendations
            f.write("=" * 70 + "\n")
            f.write("RECOMMENDATIONS\n")
            f.write("=" * 70 + "\n\n")

            f.write("For each model, the best clustering method is shown:\n\n")

            for model_name, methods in optimal_results.items():
                best_method = None
                best_score = -1

                for method, results in methods.items():
                    if results['silhouette'] > best_score:
                        best_score = results['silhouette']
                        best_method = method

                f.write(f"{model_name}:\n")
                f.write(f"  → {best_method.upper()} (Silhouette: {best_score:.4f})\n")

                if best_method == 'dbscan' or best_method == 'hdbscan': # Modified here
                    f.write(f"    {methods[best_method]['n_clusters']} clusters, "
                           f"{methods[best_method]['n_noise']} noise points\n")
                else:
                    f.write(f"    {methods[best_method]['n_clusters']} clusters\n")

                f.write(f"    Results: {methods[best_method]['txt_path'].split('/')[-1]}\n\n")

        return summary_path

    def compare_methods_across_clusters(self, model_name, cluster_range,
                                       methods=['kmeans', 'hierarchical']):
        """
        Compare different clustering methods across cluster counts

        Args:
            model_name: Embedding model to use
            cluster_range: Range of cluster counts to test
            methods: List of methods to compare (dbscan excluded as it doesn't use n_clusters)

        Returns:
            DataFrame with results and path to visualization
        """
        print(f"\n{'='*70}")
        print(f"Comparing clustering methods for {model_name}")
        print(f"{'='*70}")

        # Get embeddings
        if model_name in self.results and 'embeddings' in self.results[model_name]:
            embeddings = self.results[model_name]['embeddings']
        else:
            print(f"Generating embeddings...")
            embeddings = self.get_sentence_transformer_embeddings(model_name)
            if model_name not in self.results:
                self.results[model_name] = {}
            self.results[model_name]['embeddings'] = embeddings

        results_data = []

        # Test each method
        for method in methods:
            print(f"\nTesting {method.upper()}...")

            for n_clusters in cluster_range:
                print(f"  {n_clusters} clusters...", end=' ')

                try:
                    labels = self.cluster_embeddings(
                        embeddings,
                        n_clusters=n_clusters,
                        method=method
                    )

                    metrics = self.evaluate_clustering(embeddings, labels)

                    results_data.append({
                        'Method': method,
                        'N_Clusters': n_clusters,
                        'Silhouette': metrics['silhouette'],
                        'Davies_Bouldin': metrics['davies_bouldin'],
                        'Actual_Clusters': metrics['n_clusters']
                    })

                    print(f"Silhouette: {metrics['silhouette']:.4f}")

                except Exception as e:
                    print(f"Error: {e}")
                    continue

        # Add DBSCAN results if requested
        if 'dbscan' in methods:
            print(f"\nTesting DBSCAN (auto-tuning eps)...")
            best_labels, best_eps, dbscan_tuning = self.auto_dbscan(embeddings)

            metrics = self.evaluate_clustering(embeddings, best_labels)
            n_clusters = len(set(best_labels)) - (1 if -1 in best_labels else 0)

            results_data.append({
                'Method': 'dbscan',
                'N_Clusters': n_clusters,
                'Silhouette': metrics['silhouette'],
                'Davies_Bouldin': metrics['davies_bouldin'],
                'Actual_Clusters': n_clusters
            })

            print(f"  Best: {n_clusters} clusters (eps={best_eps:.2f}), "
                  f"Silhouette: {metrics['silhouette']:.4f}")

        df = pd.DataFrame(results_data)

        # Save to CSV
        safe_name = model_name.replace('/', '_').replace(' ', '_')
        csv_path = f'{self.output_dir}/{safe_name}_method_comparison.csv'
        df.to_csv(csv_path, index=False)

        # Create visualization
        viz_path = self._visualize_method_comparison(df, model_name)

        return df, csv_path, viz_path

    def _visualize_method_comparison(self, df, model_name):
        """Create visualization comparing clustering methods"""

        # Separate DBSCAN if present (it has only one point)
        df_methods = df[df['Method'] != 'dbscan']
        df_dbscan = df[df['Method'] == 'dbscan']

        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # Color palette
        methods = df_methods['Method'].unique()
        colors = plt.cm.Set2(np.linspace(0, 1, len(methods)))
        color_map = {method: colors[i] for i, method in enumerate(methods)}

        # Plot 1: Silhouette Score
        for method in methods:
            method_data = df_methods[df_methods['Method'] == method]
            axes[0].plot(
                method_data['N_Clusters'],
                method_data['Silhouette'],
                marker='o',
                linewidth=2.5,
                markersize=8,
                label=method.upper(),
                color=color_map[method]
            )

        # Add DBSCAN as a single point if present
        if not df_dbscan.empty:
            axes[0].scatter(
                df_dbscan['N_Clusters'],
                df_dbscan['Silhouette'],
                marker='*',
                s=400,
                color='red',
                label='DBSCAN (auto)',
                zorder=5,
                edgecolors='black',
                linewidth=1.5
            )

        axes[0].set_xlabel('Number of Clusters', fontsize=12, fontweight='bold')
        axes[0].set_ylabel('Silhouette Score', fontsize=12, fontweight='bold')
        axes[0].set_title(
            f'Silhouette Score by Clustering Method\n{model_name}',
            fontsize=13,
            fontweight='bold',
            pad=15
        )
        axes[0].legend(fontsize=10, loc='best', framealpha=0.9)
        axes[0].grid(True, alpha=0.3, linestyle='--')
        axes[0].axhline(y=0, color='gray', linestyle='-', linewidth=1, alpha=0.5)

        # Plot 2: Davies-Bouldin Index
        for method in methods:
            method_data = df_methods[df_methods['Method'] == method]
            axes[1].plot(
                method_data['N_Clusters'],
                method_data['Davies_Bouldin'],
                marker='o',
                linewidth=2.5,
                markersize=8,
                label=method.upper(),
                color=color_map[method]
            )

        # Add DBSCAN as a single point if present
        if not df_dbscan.empty:
            axes[1].scatter(
                df_dbscan['N_Clusters'],
                df_dbscan['Davies_Bouldin'],
                marker='*',
                s=400,
                color='red',
                label='DBSCAN (auto)',
                zorder=5,
                edgecolors='black',
                linewidth=1.5
            )

        axes[1].set_xlabel('Number of Clusters', fontsize=12, fontweight='bold')
        axes[1].set_ylabel('Davies-Bouldin Index', fontsize=12, fontweight='bold')
        axes[1].set_title(
            f'Davies-Bouldin Index by Clustering Method\n{model_name}',
            fontsize=13,
            fontweight='bold',
            pad=15
        )
        axes[1].legend(fontsize=10, loc='best', framealpha=0.9)
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout()

        # Save
        safe_name = model_name.replace('/', '_').replace(' ', '_')
        viz_path = f'{self.output_dir}/{safe_name}_method_comparison.png'
        plt.savefig(viz_path, dpi=150, bbox_inches='tight')
        plt.close()

        print(f"\nVisualization saved: {viz_path}")

        return viz_path

    def visualize_cluster_comparison(self, df):
        """
        Create line plots comparing models across cluster counts

        Args:
            df: DataFrame with columns: Model, N_Clusters, Silhouette, Davies_Bouldin

        Returns:
            Path to saved visualization
        """
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))

        # Color palette for models
        models = df['Model'].unique()
        colors = plt.cm.Set2(np.linspace(0, 1, len(models)))
        color_map = {model: colors[i] for i, model in enumerate(models)}

        # Plot 1: Silhouette Score vs Cluster Count
        for model in models:
            model_data = df[df['Model'] == model]
            axes[0].plot(
                model_data['N_Clusters'],
                model_data['Silhouette'],
                marker='o',
                linewidth=2,
                label=model.split('/')[-1],  # Use short name
                color=color_map[model]
            )

        axes[0].set_xlabel('Number of Clusters', fontsize=11)
        axes[0].set_ylabel('Silhouette Score', fontsize=11)
        axes[0].set_title('Silhouette Score vs Cluster Count\n(higher is better)',
                         fontsize=12, fontweight='bold')
        axes[0].legend(fontsize=9, loc='best')
        axes[0].grid(True, alpha=0.3)
        axes[0].axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)

        # Plot 2: Davies-Bouldin Index vs Cluster Count
        for model in models:
            model_data = df[df['Model'] == model]
            axes[1].plot(
                model_data['N_Clusters'],
                model_data['Davies_Bouldin'],
                marker='o',
                linewidth=2,
                label=model.split('/')[-1],
                color=color_map[model]
            )

        axes[1].set_xlabel('Number of Clusters', fontsize=11)
        axes[1].set_ylabel('Davies-Bouldin Index', fontsize=11)
        axes[1].set_title('Davies-Bouldin Index vs Cluster Count\n(lower is better)',
                         fontsize=12, fontweight='bold')
        axes[1].legend(fontsize=9, loc='best')
        axes[1].grid(True, alpha=0.3)

        # Plot 3: Combined Score (normalized)
        # Normalize both metrics to 0-1 scale where better = higher
        for model in models:
            model_data = df[df['Model'] == model].copy()

            # Normalize silhouette (already -1 to 1, shift to 0-1)
            norm_silhouette = (model_data['Silhouette'] + 1) / 2

            # Normalize Davies-Bouldin (invert and scale)
            # Lower DB is better, so we invert it
            max_db = df['Davies_Bouldin'].max()
            min_db = df['Davies_Bouldin'].min()
            if max_db > min_db:
                norm_db = 1 - (model_data['Davies_Bouldin'] - min_db) / (max_db - min_db)
            else:
                norm_db = model_data['Davies_Bouldin'] * 0  # All same, set to 0

            # Combined score (average of normalized metrics)
            combined = (norm_silhouette + norm_db) / 2

            axes[2].plot(
                model_data['N_Clusters'],
                combined,
                marker='o',
                linewidth=2,
                label=model.split('/')[-1],
                color=color_map[model]
            )

        axes[2].set_xlabel('Number of Clusters', fontsize=11)
        axes[2].set_ylabel('Combined Score', fontsize=11)
        axes[2].set_title('Combined Quality Score\n(higher is better)',
                         fontsize=12, fontweight='bold')
        axes[2].legend(fontsize=9, loc='best')
        axes[2].grid(True, alpha=0.3)
        axes[2].set_ylim([0, 1])

        plt.tight_layout()

        # Save figure
        viz_path = f'{self.output_dir}/cluster_count_comparison.png'
        plt.savefig(viz_path, dpi=150, bbox_inches='tight')
        plt.close()

        return viz_path

    def create_dendrogram(self, embeddings, model_name, max_display=50):
        """
        Create a dendrogram visualization for hierarchical clustering

        Args:
            embeddings: Embedding vectors
            model_name: Name of the model
            max_display: Maximum number of leaf nodes to display

        Returns:
            Path to saved dendrogram
        """
        from scipy.cluster.hierarchy import linkage, dendrogram
        from scipy.spatial.distance import squareform
        from sklearn.metrics.pairwise import cosine_distances

        print(f"Creating dendrogram for {model_name}...")

        # Compute distance matrix
        distances = cosine_distances(embeddings)
        condensed_distances = squareform(distances)

        # Perform hierarchical clustering
        Z = linkage(condensed_distances, method='average')

        # Create figure
        fig, ax = plt.subplots(figsize=(20, 10))

        # Plot dendrogram
        dendrogram(
            Z,
            ax=ax,
            truncate_mode='lastp',
            p=max_display,
            leaf_font_size=10,
            show_contracted=True
        )

        ax.set_title(
            f'Hierarchical Clustering Dendrogram - {model_name}\n'
            f'(Showing top {max_display} clusters)',
            fontsize=14,
            fontweight='bold'
        )
        ax.set_xlabel('Cluster Index or (Sample Count)', fontsize=12)
        ax.set_ylabel('Cosine Distance', fontsize=12)

        plt.tight_layout()

        # Save figure
        safe_name = model_name.replace('/', '_').replace(' ', '_')
        filename = f'{safe_name}_dendrogram.png'
        filepath = f'{self.output_dir}/{filename}'
        plt.savefig(filepath, dpi=150, bbox_inches='tight')
        plt.close()

        return filepath

    def export_cluster_members(self, model_name, labels, n_samples_per_cluster=20):
        """
        Export cluster members to CSV and text files

        Args:
            model_name: Name of the model
            labels: Cluster labels for each word
            n_samples_per_cluster: Number of sample words to show per cluster in summary

        Returns:
            Paths to generated files
        """
        import pandas as pd

        print(f"Exporting cluster members for {model_name}...")

        # Create dataframe
        df = pd.DataFrame({
            'word': self.words,
            'cluster': labels
        })

        # Sort by cluster
        df = df.sort_values('cluster')

        # Save full CSV
        safe_name = model_name.replace('/', '_').replace(' ', '_')
        csv_path = f'{self.output_dir}/{safe_name}_clusters_full.csv'
        df.to_csv(csv_path, index=False)

        # Create summary text file
        txt_path = f'{self.output_dir}/{safe_name}_clusters_summary.txt'

        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(f"Cluster Summary for {model_name}\n")
            f.write("=" * 70 + "\n\n")

            unique_clusters = sorted(df['cluster'].unique())

            # Handle noise points in DBSCAN
            if -1 in unique_clusters:
                noise_words = df[df['cluster'] == -1]['word'].tolist()
                f.write(f"NOISE POINTS (not assigned to any cluster): {len(noise_words)} words\n")
                f.write(f"Sample: {', '.join(noise_words[:20])}\n")
                if len(noise_words) > 20:
                    f.write(f"... and {len(noise_words) - 20} more\n")
                f.write("\n" + "=" * 70 + "\n\n")
                unique_clusters = [c for c in unique_clusters if c != -1]

            # Write each cluster
            for cluster_id in unique_clusters:
                cluster_words = df[df['cluster'] == cluster_id]['word'].tolist()

                f.write(f"CLUSTER {cluster_id}: {len(cluster_words)} words\n")
                f.write("-" * 70 + "\n")

                # Show sample
                sample = cluster_words[:n_samples_per_cluster]
                f.write(f"Sample ({len(sample)} of {len(cluster_words)}): ")
                f.write(", ".join(sample))

                if len(cluster_words) > n_samples_per_cluster:
                    f.write(f"\n... and {len(cluster_words) - n_samples_per_cluster} more")

                f.write("\n\n")

            # Statistics
            f.write("=" * 70 + "\n")
            f.write("STATISTICS\n")
            f.write("=" * 70 + "\n")
            f.write(f"Total words: {len(self.words)}\n")
            f.write(f"Number of clusters: {len(unique_clusters)}\n")

            if len(unique_clusters) > 0:
                cluster_sizes = df[df['cluster'] != -1].groupby('cluster').size()
                f.write(f"Average cluster size: {cluster_sizes.mean():.1f}\n")
                f.write(f"Largest cluster: {cluster_sizes.max()} words\n")
                f.write(f"Smallest cluster: {cluster_sizes.min()} words\n")

        print(f"  Saved full data: {csv_path}")
        print(f"  Saved summary: {txt_path}")

        return csv_path, txt_path

    def create_comparison_report(self):
        """Create a summary report comparing all models"""
        # Collect metrics
        comparison_data = []

        for model_name, results in self.results.items():
            for viz in results['visualizations']:
                comparison_data.append({
                    'Model': model_name,
                    'Reduction': viz['reduction'],
                    'Clustering': viz['clustering'],
                    'N Clusters': viz['metrics']['n_clusters'],
                    'Silhouette': viz['metrics']['silhouette'],
                    'Davies-Bouldin': viz['metrics']['davies_bouldin']
                })

        df = pd.DataFrame(comparison_data)

        # Create comparison visualization
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # Silhouette scores
        if df['Silhouette'].notna().any():
            silhouette_data = df.dropna(subset=['Silhouette'])
            silhouette_pivot = silhouette_data.pivot_table(
                values='Silhouette',
                index='Model',
                columns='Reduction',
                aggfunc='mean'
            )
            sns.heatmap(silhouette_pivot, annot=True, fmt='.3f',
                       cmap='RdYlGn', ax=axes[0], vmin=-1, vmax=1)
            axes[0].set_title('Silhouette Scores (higher is better)', fontweight='bold')

        # Davies-Bouldin scores
        if df['Davies-Bouldin'].notna().any():
            db_data = df.dropna(subset=['Davies-Bouldin'])
            db_pivot = db_data.pivot_table(
                values='Davies-Bouldin',
                index='Model',
                columns='Reduction',
                aggfunc='mean'
            )
            sns.heatmap(db_pivot, annot=True, fmt='.3f',
                       cmap='RdYlGn_r', ax=axes[1])
            axes[1].set_title('Davies-Bouldin Scores (lower is better)', fontweight='bold')

        plt.tight_layout()
        comparison_path = f'{self.output_dir}/model_comparison.png'
        plt.savefig(comparison_path, dpi=150, bbox_inches='tight')
        plt.close()

        # Save metrics table
        csv_path = f'{self.output_dir}/comparison_metrics.csv'
        df.to_csv(csv_path, index=False)

        return comparison_path, csv_path, df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
%%time
"""
Quick Start Script for Embedding Comparison
Load your wordplay terms and run the comparison
"""

def load_words_from_file(filepath):
    """
    Load words from various file formats
    Supports: txt (one word per line), csv, excel
    """
    if filepath.endswith('.txt'):
        with open(filepath, 'r', encoding='utf-8') as f:
            words = [line.strip() for line in f if line.strip()]
    elif filepath.endswith('.csv'):
        df = pd.read_csv(filepath)
        # Assumes words are in first column, adjust if needed
        words = df.iloc[:, 0].dropna().astype(str).tolist()
    elif filepath.endswith(('.xlsx', '.xls')):
        df = pd.read_excel(filepath)
        words = df.iloc[:, 0].dropna().astype(str).tolist()
    else:
        raise ValueError("Unsupported file format. Use .txt, .csv, or .xlsx")

    # NC, drop duplicate indicators, lose signal on frequent indicators
    # words = list(set(words))
    return words

def run_comparison(words, n_clusters=10, models=None):
    """
    Run the full comparison pipeline

    Args:
        words: List of wordplay terms
        n_clusters: Number of clusters for k-means (adjust based on your data)
        models: List of model names to compare (uses defaults if None)
    """

    # Default models (fast to very good quality)
    if models is None:
        models = [
            'all-MiniLM-L6-v2',          # 384 dim, very fast
            'all-mpnet-base-v2',         # 768 dim, high quality
            'all-MiniLM-L12-v2',         # 384 dim, balanced
        ]

    print(f"\nAnalyzing {len(words)} unique wordplay terms")
    print(f"Using {len(models)} embedding models")
    print(f"Target clusters: {n_clusters}\n")

    # Initialize
    comparator = EmbeddingComparison(words)

    # Analyze each model
    for i, model in enumerate(models, 1):
        print(f"\n[{i}/{len(models)}] Processing {model}...")
        try:
            comparator.analyze_model(
                model,
                reduction_methods=['umap', 'tsne'],
                clustering_methods=['kmeans'],
                n_clusters=n_clusters
            )
        except Exception as e:
            print(f"Error with {model}: {e}")
            continue

    # Create comparison report
    if comparator.results:
        print("\nGenerating comparison report...")
        comp_path, csv_path, metrics_df = comparator.create_comparison_report()

        print("\n" + "="*70)
        print("ANALYSIS COMPLETE!")
        print("="*70)
        print(f"\nResults saved to: {comparator.output_dir}/")
        print("\nFiles generated:")
        for model in comparator.results.keys():
            print(f"\n  {model}:")
            for viz in comparator.results[model]['visualizations']:
                print(f"    - {viz['path'].split('/')[-1]}")
        print(f"\n  Comparison:")
        print(f"    - {comp_path.split('/')[-1]}")
        print(f"    - {csv_path.split('/')[-1]}")

        print("\n" + "="*70)
        print("Metrics Summary:")
        print("="*70)
        print(metrics_df.to_string())

        return comparator
    else:
        print("\nNo results generated. Check for errors above.")
        return None


def run_cluster_count_comparison(words, cluster_range=None, models=None):
    """
    Compare models across different cluster counts

    Args:
        words: List of wordplay terms
        cluster_range: List or range of cluster counts to test (default: 5 to 30 by 5)
        models: List of model names to compare (uses defaults if None)

    Returns:
        EmbeddingComparison object with results
    """

    # Default models
    if models is None:
        models = [
            'all-MiniLM-L6-v2',
            'all-mpnet-base-v2',
            'all-MiniLM-L12-v2',
        ]

    # Default cluster range
    if cluster_range is None:
        cluster_range = range(5, 31, 5)  # 5, 10, 15, 20, 25, 30

    print(f"\nComparing cluster counts for {len(words)} wordplay terms")
    print(f"Models: {len(models)}")
    print(f"Cluster counts to test: {list(cluster_range)}\n")

    # Initialize
    comparator = EmbeddingComparison(words)

    # Run comparison
    df_results, csv_path, viz_path = comparator.compare_cluster_counts(
        model_names=models,
        cluster_range=cluster_range,
        reduction_method='umap'
    )

    print("\n" + "="*70)
    print("CLUSTER COUNT COMPARISON COMPLETE!")
    print("="*70)
    print(f"\nResults saved to: {comparator.output_dir}/")
    print(f"  - Visualization: {viz_path.split('/')[-1]}")
    print(f"  - Data: {csv_path.split('/')[-1]}")

    # Print best configurations
    print("\n" + "="*70)
    print("Best Cluster Count per Model (by Silhouette Score):")
    print("="*70)

    for model in models:
        model_data = df_results[df_results['Model'] == model]
        best_idx = model_data['Silhouette'].idxmax()
        best_row = model_data.loc[best_idx]

        print(f"\n{model.split('/')[-1]}:")
        print(f"  Optimal clusters: {int(best_row['N_Clusters'])}")
        print(f"  Silhouette: {best_row['Silhouette']:.4f}")
        print(f"  Davies-Bouldin: {best_row['Davies_Bouldin']:.4f}")

    return comparator, df_results


def run_full_optimal_analysis(words, cluster_range=None, models=None,
                              methods=['kmeans', 'dbscan', 'hierarchical']):
    """
    Complete analysis: find optimal clusters and export members for all methods

    This is the recommended function for comprehensive analysis!

    Args:
        words: List of wordplay terms
        cluster_range: Range of cluster counts to test for kmeans/hierarchical
        models: List of model names to compare (uses defaults if None)
        methods: Clustering methods to try (default: all three)

    Returns:
        EmbeddingComparison object and optimal results dictionary
    """

    # Default models
    if models is None:
        models = [
            'all-MiniLM-L6-v2',    # Fast
            'all-mpnet-base-v2',   # High quality
        ]

    # Default cluster range
    if cluster_range is None:
        cluster_range = range(5, 31, 5)

    print("\n" + "="*70)
    print("COMPREHENSIVE OPTIMAL CLUSTERING ANALYSIS")
    print("="*70)
    print(f"\nWords: {len(words)}")
    print(f"Models: {', '.join(models)}")
    print(f"Methods: {', '.join(methods)}")
    print(f"Cluster range: {list(cluster_range)}")
    print("\nThis will:")
    print("  1. Test each model with multiple clustering methods")
    print("  2. Find optimal parameters for each method")
    print("  3. Export cluster members to CSV and text files")
    print("  4. Create dendrograms for hierarchical clustering")
    print("  5. Generate comparison visualizations")

    # Initialize
    comparator = EmbeddingComparison(words)

    # Run comprehensive analysis
    optimal_results = comparator.find_and_export_optimal_clusters(
        model_names=models,
        cluster_range=cluster_range,
        methods=methods
    )

    print("\n" + "="*70)
    print("ANALYSIS COMPLETE! Generated files:")
    print("="*70)

    for model_name, method_results in optimal_results.items():
        print(f"\n{model_name}:")
        for method, results in method_results.items():
            print(f"  {method.upper()}:")
            print(f"    - {results['txt_path'].split('/')[-1]} (summary)")
            print(f"    - {results['csv_path'].split('/')[-1]} (full data)")
            if 'dendrogram_path' in results:
                print(f"    - {results['dendrogram_path'].split('/')[-1]} (dendrogram)")

    print(f"\n  Summary: optimal_clusters_summary.txt")

    return comparator, optimal_results


def compare_clustering_methods(words, cluster_range=None, model='all-mpnet-base-v2',
                               methods=['kmeans', 'hierarchical', 'dbscan']):
    """
    Compare different clustering methods across cluster counts for a single model

    Generates a plot showing how silhouette scores vary with cluster count
    for each clustering method.

    Args:
        words: List of wordplay terms
        cluster_range: Range of cluster counts to test (default: 5 to 30 by 5)
        model: Which embedding model to use
        methods: Which clustering methods to compare

    Returns:
        EmbeddingComparison object and results DataFrame
    """

    if cluster_range is None:
        cluster_range = range(5, 31, 5)

    print("\n" + "="*70)
    print("CLUSTERING METHOD COMPARISON")
    print("="*70)
    print(f"\nModel: {model}")
    print(f"Methods: {', '.join(methods)}")
    print(f"Cluster range: {list(cluster_range)}")
    print("\nThis will generate:")
    print("  • 2-panel plot comparing methods")
    print("  • CSV with all metrics")

    # Initialize
    comparator = EmbeddingComparison(words)

    # Run comparison
    df_results, csv_path, viz_path = comparator.compare_methods_across_clusters(
        model_name=model,
        cluster_range=cluster_range,
        methods=methods
    )

    print("\n" + "="*70)
    print("METHOD COMPARISON COMPLETE!")
    print("="*70)
    print(f"\nResults:")
    print(f"  Visualization: {viz_path.split('/')[-1]}")
    print(f"  Data: {csv_path.split('/')[-1]}")

    # Print summary
    print("\n" + "="*70)
    print("Best Performance by Method:")
    print("="*70)

    for method in methods:
        method_data = df_results[df_results['Method'] == method]

        if len(method_data) == 0:
            continue

        best_idx = method_data['Silhouette'].idxmax()
        best_row = method_data.loc[best_idx]

        print(f"\n{method.upper()}:")
        if method == 'dbscan':
            print(f"  Clusters found: {int(best_row['N_Clusters'])}")
        else:
            print(f"  Optimal clusters: {int(best_row['N_Clusters'])}")
        print(f"  Silhouette: {best_row['Silhouette']:.4f}")
        print(f"  Davies-Bouldin: {best_row['Davies_Bouldin']:.4f}")

    return comparator, df_results


# EXAMPLE USAGE OPTIONS:

# Option 1: Load from file
# words = load_words_from_file('/mnt/user-data/uploads/your_wordplay_terms.txt')
# run_comparison(words, n_clusters=15)
words = load_words_from_file(IND_DIR)

# Option 3: Custom models
# custom_models = [
#     'all-MiniLM-L6-v2',
#     'paraphrase-multilingual-mpnet-base-v2',
#     'sentence-transformers/all-roberta-large-v1'
# ]
# run_comparison(words, n_clusters=12, models=custom_models)

# Option 4: Compare across cluster counts
# words = load_words_from_file('/mnt/user-data/uploads/your_wordplay_terms.txt')
# comparator, results_df = run_cluster_count_comparison(
#     words,
#     cluster_range=range(5, 51, 5),  # Test 5, 10, 15, ... 50
#     models=['all-MiniLM-L6-v2', 'all-mpnet-base-v2']
# )

# Option 5: COMPREHENSIVE ANALYSIS (RECOMMENDED!)
# This finds optimal clusters and exports members for all methods
# words = load_words_from_file('/mnt/user-data/uploads/your_wordplay_terms.txt')
comparator, optimal_results = run_full_optimal_analysis(
    words,
    cluster_range=range(10, 51, 5),
    models=[#'all-MiniLM-L6-v2',
            'all-mpnet-base-v2'
            ],
    methods=[#'kmeans',
             #'dbscan',
             'hdbscan'
             #'hierarchical'
             ]
)

# Option 6: COMPARE CLUSTERING METHODS (NEW!)
# Compare K-means vs Hierarchical vs DBSCAN on one model
# words = load_words_from_file('/mnt/user-data/uploads/your_wordplay_terms.txt')
#comparator, results_df = compare_clustering_methods(
#    words,
#   cluster_range=range(5, 51, 5),
#    model='all-mpnet-base-v2',
#    methods=['kmeans',
#             'hierarchical',
#             'dbscan'
#             ]
#)

if __name__ == "__main__":
    print("="*70)
    print("EMBEDDING MODEL COMPARISON FOR WORDPLAY CLUSTERING")
    print("="*70)
    print("\nTo use this script:")
    print("2. Uncomment and modify one of the example usage options above")
    print("3. Run the script")
    print("="*70)


COMPREHENSIVE OPTIMAL CLUSTERING ANALYSIS

Words: 14195
Models: all-mpnet-base-v2
Methods: hdbscan
Cluster range: [10, 15, 20, 25, 30, 35, 40, 45, 50]

This will:
  1. Test each model with multiple clustering methods
  2. Find optimal parameters for each method
  3. Export cluster members to CSV and text files
  4. Create dendrograms for hierarchical clustering
  5. Generate comparison visualizations

Finding optimal clusters for all-mpnet-base-v2
  Generating embeddings...
Loading all-mpnet-base-v2...


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Generating embeddings for 14195 words...


Batches:   0%|          | 0/444 [00:00<?, ?it/s]


  Testing HDBSCAN clustering...
Min size:10, min samples:1
Min size:10, min samples:5
Min size:10, min samples:10
Min size:15, min samples:1
Min size:15, min samples:5
Min size:15, min samples:10
Min size:20, min samples:1
Min size:20, min samples:5
Min size:20, min samples:10
  ✓ HDBSCAN optimal: min_cluster_size=15, min_samples=10 (Silhouette: -0.2345)
Exporting cluster members for all-mpnet-base-v2_hdbscan_15_10...
  Saved full data: /content/drive/MyDrive/SIADS 692 Milestone II/Milestone II - NLP Cryptic Crossword Clues/outputs/all-mpnet-base-v2_hdbscan_15_10_clusters_full.csv
  Saved summary: /content/drive/MyDrive/SIADS 692 Milestone II/Milestone II - NLP Cryptic Crossword Clues/outputs/all-mpnet-base-v2_hdbscan_15_10_clusters_summary.txt

OPTIMAL CLUSTER ANALYSIS COMPLETE!

Summary report: /content/drive/MyDrive/SIADS 692 Milestone II/Milestone II - NLP Cryptic Crossword Clues/outputs/optimal_clusters_summary.txt

ANALYSIS COMPLETE! Generated files:

all-mpnet-base-v2:
  HDBSCA